# Understanding Aqua's Operator Flow

_donny@, 30-Apr-20_

<h2>Table of Contents<span class="tocSkip"></span></h2>
<div class="toc"><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Overview</a></span><ul class="toc-item"><li><span><a href="#Basic-Design-Principles" data-toc-modified-id="Basic-Design-Principles-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Basic Design Principles</a></span></li><li><span><a href="#Basic-Definitions" data-toc-modified-id="Basic-Definitions-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Basic Definitions</a></span><ul class="toc-item"><li><span><a href="#Chaining-vs.-Composition" data-toc-modified-id="Chaining-vs.-Composition-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Chaining vs. Composition</a></span></li></ul></li><li><span><a href="#Object-Structure" data-toc-modified-id="Object-Structure-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Object Structure</a></span></li><li><span><a href="#Motivating-Example" data-toc-modified-id="Motivating-Example-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Motivating Example</a></span></li></ul></li><li><span><a href="#Part-I:-State-Functions-and-Measurements" data-toc-modified-id="Part-I:-State-Functions-and-Measurements-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Part I: State Functions and Measurements</a></span></li><li><span><a href="#Part-II:-PrimitiveOps" data-toc-modified-id="Part-II:-PrimitiveOps-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Part II: PrimitiveOps</a></span></li><li><span><a href="#Part-III:-ListOps" data-toc-modified-id="Part-III:-ListOps-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Part III: ListOps</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Hierarchical-ListOps" data-toc-modified-id="Hierarchical-ListOps-4.0.1"><span class="toc-item-num">4.0.1&nbsp;&nbsp;</span>Hierarchical ListOps</a></span></li></ul></li></ul></li><li><span><a href="#Part-IV:-Converters" data-toc-modified-id="Part-IV:-Converters-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Part IV: Converters</a></span><ul class="toc-item"><li><span><a href="#Evolutons,-exp_i()-and-the-EvolvedOp" data-toc-modified-id="Evolutons,-exp_i()-and-the-EvolvedOp-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Evolutons, <code>exp_i()</code> and the <code>EvolvedOp</code></a></span></li><li><span><a href="#Expectations-for-Converting-Non-diagonal-Measurements" data-toc-modified-id="Expectations-for-Converting-Non-diagonal-Measurements-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Expectations for Converting Non-diagonal Measurements</a></span></li><li><span><a href="#Executing-CircuitStateFns-with-the-CircuitSampler" data-toc-modified-id="Executing-CircuitStateFns-with-the-CircuitSampler-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Executing <code>CircuitStateFn</code>s with the CircuitSampler</a></span></li></ul></li></ul></div>

## Overview

A library for Quantum Algorithms & Applications is more than a collection of procedural code wrapped in Python functions. It needs to provide tools to make writing algorithms simple and easy. This is the layer of modules between the circuits and algorithms, providing the language and computational primitives for Quantum algorithms research.

In Aqua, we call this layer the Operator flow. It works by unifying computation with theory through the common language of functions and operators in a way which preserves physical intuition and programming freedom. In the Operator flow, we construct functions over binary variables, manipulate those functions with operators, and evaluate properties of these functions with measurements. 

Below, we'll describe the key players in the Operator flow, and show some examples of how they can be used to construct some familiar quantum algorithms. This notebook assumes a basic familiarity with Quantum Algorithms.

**Note: The Operator flow is a large and self-referential system. Do not worry if you don't immediately feel comfortable with all of its moving parts - like any system of abstractions, it can take time and play to feel comfortable. We feel that this is a worthwhile investment, as once you begin to feel comfortable with the system, algorithm development is super fast and easy.**

### Basic Design Principles

* *Batteries Included* - We prioritize quick and easy access to the tools you're most likely to reach for when prototyping.

* *Physically Formal* - The Operator flow was built to be mathematically formal down to its very bottom primitives, so that it may serve as a lingua franca between the theory and implementation of Quantum algorithms.

* *Powerful OR Readable* - We've tried to let any operation be as succinct or verbose as you want it to be, including both rich syntactic sugar and long-form interfaces.

* *Fast by Default* - We use thoughtfully selected defaults so that algorithms run as fast as possible at many scales, and so you only need to toggle the settings you care about. We want you to be able to trust that we're running your problem as fast as we can, and if you find a way to run it faster, you should [let us know!](https://github.com/Qiskit/qiskit-aqua/issues)

### Basic Definitions

Before getting into the details of the code, it's important to note that three mathematical concepts unpin the Operator flow. We derive most of the inspiration for the code structure from [John Watrous's Quantum Information formalism](https://cs.uwaterloo.ca/~watrous/TQI/) (but do not follow it exactly), so it may be worthwhile to review Chapters I and II, which are free online, if you feel the concepts are not clicking.

1. **"State functions"** - An $n$-qubit State function is a complex function over $n$ binary variables, which we will often refer to as _$n$-qubit binary strings_. For example, the traditional quantum "zero state" is a 1-qubit state function, with a definition of $Zero(0) = 1$ and $Zero(1) = 0$. It is often convenient to think about State functions as traditional kets (i.e. $Zero = |0\rangle$), but be warned, **State functions need not be normalized.** They are just functions.

1. **"Operators"** - An $n$-qubit Operator is a linear function taking $n$-qubit state functions to $n$-qubit state functions. For example, the Pauli X Operator is defined by $X(Zero) = One$ and $X(One) = Zero$. Therefore, $X(Zero)(0) = 0$. Equivalently, an Operator can be defined as a complex function over two n-qubit binary strings, and it is sometimes convenient to picture things this way. By this definition, our Pauli $X$ can be defined by its typical matrix elements, $X(0, 0) = 0$, $X(1, 0) = 1$, $X(0, 1) = 1$, $X(1, 1) = 0$. In the Operator flow, our Operators are all binary, square Operators (for now).

1. **"Measurements"** - An $n-qubit Measurement$ is a functional taking n-qubit State functions to complex values. For example, a Pauli Z Measurement can be defined by $Z_{meas}(Zero) = 1$ and $Z_{meas}(One) = -1$. Measurements are simply adjoints of State Functions, and can be imagined by quantum mechanics' _bra_ notation (i.e. $Zero_{meas} = Zero^{\dagger} = \langle 0|$). In thinking of Measurements as adjoints of State functions, note that $Z_{meas}$ above is equivalent to $1 - 2*(\langle Zero| + \langle One|)$.

**Throughout this tutorial, we will refer to the mathematical function (of type 1-3 above) that a Python Object represents as that Object's *characteristic function*.**

#### Chaining vs. Composition
Note that a chained expression of the functions above can be written in two ways:
* As function chaining: $Z(X(Zero))(0) = 0$
* As function composition: $Zero_{meas} \circ Z \circ X \circ Zero = 0$

In the Operator flow, chaining is used via the `.eval()` method, while composition is used via `.compose()` or `@` (more on this below).

### Object Structure

The Operator flow includes two primary groups of actors: 
* Operators, objects which represent functions or functionals, all deriving from `OperatorBase`: 
    * `PrimitiveOp`s - Basic Operator building blocks, whose behavior are defined by some internal computational primitive from Terra, such as a Terra's `Pauli` (`PauliOp`), `Operator` (`MatrixOp`), or `QuantumCircuit` (`CircuitOp`).
    * `StateFn`s - A class for representing both State functions and Measurements, the behavior of which is defined by some internal primitive, e.g. a simple dict (`DictStateFn`), Terra's `Statevector` (`VectorStateFn`), a `QuantumCircuit` (`CircuitStateFn`), or an OperatorBase representing a density operator (`OperatorStateFn`).
    * `ListOp`s - Classes for representing composite Operators, State functions, and Measurements constructed from expressions of others, such as addition (`SummedOp`), composition (`ComposedOp`), tensor product (`TensoredOp`), and concatination into a list (`ListOp`).
    * `EvolvedOp` - A special case of a `PrimitiveOp` holding an `Operatorbase` as its primitive, serving as a placeholder for an evolution algorithm to convert later into an Operator approximating the exponentiation of the `OperatorBase`.
* Converters, objects which manipulate Operators, all deriving from `ConverterBase`:
    * `Expectation`s - Traverse over an Operator and replace OperatorStateFn measurements into Z-basis or matrix measurements approximating the expectation value of the measurement.
    * `Evolution`s - Traverse over an Operator and replace `EvolvedOp`s with circuits approximating the evolution.
    * `CircuitSampler` - Traverse over an Operator and replace `CircuitStateFn`s with `DictStateFn`s approximating them, perhaps sampled from a quantum device.
    * Other converters, e.g. `AbelianGrouper`, `PauliBasisChange`
    
Don't worry if this seems like a whirlwind. We'll discuss each of these in detail in the next section, and you can also find more detail in the [Operator flow API reference](https://qiskit.org/documentation/apidoc/qiskit.aqua.operators.html#module-qiskit.aqua.operators).

### Motivating Example

Below, we'll show some basic code to highlight the power of the Operator flow. The code is simple - we'll evolve a Bell state by an H2 Hamiltonian using a simple Trotter expansion and measure the energy expectation value of the Hamiltonian at various evolution times. You are obviously not expected to understand it, but inspect it to gain some context and see how it simplifies otherwise complex constructions. Also note that this is a slightly verbose form of this flow for instruction's sake.

In the Converters section of this notebook, we'll walk through the example in more detail.

In [1]:
from qiskit import BasicAer
from qiskit.circuit import Parameter
from qiskit.aqua.operators import *
import numpy as np

two_qubit_H2 =  (-1.0523732 * I^I) + \
                (0.39793742 * I^Z) + \
                (-0.3979374 * Z^I) + \
                (-0.0112801 * Z^Z) + \
                (0.18093119 * X^X)

bell = CX @ (Plus^Zero)
evo_time = Parameter('θ')
evolution_op = (evo_time*two_qubit_H2).exp_i()
h2_measurement = StateFn(two_qubit_H2).adjoint()
evo_and_meas = h2_measurement @ evolution_op @ bell

trotterized_op = PauliTrotterEvolution(trotter_mode=Suzuki()).convert(evo_and_meas)
diagonalized_meas_op = PauliExpectation().convert(trotterized_op)
evo_time_points = list(range(8))
h2_trotter_expectations = diagonalized_meas_op.bind_parameters({evo_time: evo_time_points})

# Energies by Trotter expansion and sampled measurement
sampler = CircuitSampler(backend=BasicAer.get_backend('qasm_simulator'))
sampled_trotter_exp_op = sampler.convert(h2_trotter_expectations)
sampled_trotter_energies = sampled_trotter_exp_op.eval()
print('Sampled Trotterized energies:\n {}'.format(np.real(sampled_trotter_energies)))

Sampled Trotterized energies:
 [-1.05590701 -1.03717781 -1.03293723 -1.04530558 -1.05873406 -1.07781665
 -1.09053837 -1.08947823  0.38514937  0.39247484  0.38017148  0.39411741
  0.38275161  0.37542002  0.28899421  0.17760919]


## Part I: State Functions and Measurements

The most basic unit of computation in the Operator flow is the state function, abbreviated as `StateFn`, a simple complex function over binary variables. There are several `StateFn` instances built in for convenience, `Zero, One, Plus, Minus`, and four ways a `StateFn` can be defined, `DictStateFn, VectorStateFn, CircuitStateFn, OperatorStateFn`.

Every `StateFn` has three properties:
* `primitive` - The data structure defining the underlying function's behavior. For `DictStateFn`, this object is a `dict`, etc.
* `coeff` - A coefficient multiplying the characteristic function, i.e. `StateFn(my_primitive, coeff=3) == StateFn(my_primitive) * 3`. Note that `coeff` can be int, float, complex or a free `Parameter` object (from `qiskit.circuit` in Terra) to be bound later.
* `is_measurement` - Whether this `StateFn` is a state function or a measurement (remember, a measurement is just the adjoint of a state function).

In [2]:
from qiskit.aqua.operators import (StateFn, Zero, One, Plus, Minus, 
                                   DictStateFn, VectorStateFn, CircuitStateFn, OperatorStateFn)

In [3]:
print(Zero)
print(Zero.adjoint())

DictStateFn({'0': 1})
DictMeasurement({'0': 1})


The function this object represents, mapping a binary string to a complex value, can be accessed via the `.eval` method.

In [4]:
print(Zero.eval('0'))
print(Zero.eval('1'))
print(One.eval('1'))
print(Plus.eval('0'))
print(Minus.eval('1'))

1.0
0.0
1.0
(0.70710678118655+0j)
(-0.70710678118655+0j)


This should look familiar as something analogous to the quantum state or wavefunctions you know, $|0\rangle$, $|1\rangle$, $|+\rangle$, and $|-\rangle$, but with a key difference: these statefunctions need not be normalized. `statefunction1 + statefunction2` behaves exactly as you'd expect mathematical functions to behave, producing `statefunction3`, where `statefunction3(x) = statefunction1(x) + statefunction2(x)`.

In [5]:
print((One + One + One).eval('1'))
print((One + One + One))
print((Plus + Minus).eval('0'))
print((Plus + Minus).eval('1'))

print(~Plus @ Zero)

3.0
DictStateFn({'1': 1}) * 3.0
(1.4142135623731+0j)
0j
ComposedOp(
[CircuitMeasurement(
     ┌───┐
q_0: ┤ H ├
     └───┘
),
DictStateFn({'0': 1})])


The behavior of each StateFn type is defined internally by some data structure, which we call the primitive, and a complex coefficient. The `DictStateFn` is the simplest type, holding a dict primitive of {string: complex} value pairs. `Zero` and `One` are simply these. Any value missing from the dict is simply equal to 0.

In [6]:
print(Zero)
print(One)
print(Zero.primitive)
print(Zero.coeff)
print((Zero + Zero + Zero).coeff)
print((Zero * 2j).coeff)

DictStateFn({'0': 1})
DictStateFn({'1': 1})
{'0': 1}
1.0
3.0
2j


For simplicity, the `StateFn` class also doubles as the class for Measurement. The adjoint of a `StateFn` is simply a measurement defined by the same primitive as the state. This is conceptually identical to the idea that a bra is the adjoint of a ket, and it is convenient to think of it that way.

Just as the State function over binary variables is accessible via `.eval`, so to the Measurement functional over State functions is available via `.eval`. As we'll see below, the availability of the underlying function through `.eval` is also true for the Operators.

In [7]:
print(One.adjoint())
print(Zero.adjoint().eval(One))
print(Zero.adjoint().eval(Zero))
print(Zero.adjoint().eval(Plus))

DictMeasurement({'1': 1})
0.0
1.0
(0.70710678118655+0j)


Note that we can also perform function composition between a State function and measure using `.compose`, and call `.eval()` with no argument later.

In [8]:
print(Zero.adjoint().compose(One).eval())
print(Zero.adjoint().compose(Zero).eval())
print(Zero.adjoint().compose(Plus).eval())

0.0
1.0
(0.70710678118655+0j)


Nearly all arithmetic operations between StateFns are supported, including:
* `+` - addition
* `-` - subtraction, negation (scalar multiplication by -1)
* `*` - scalar multiplication
* `/` - scalar division
* `@` - composition
* `^` - tensor product or tensor power (tensor with self n times)
* `**` - composition power (compose with self n times)
* `==` - equality
* `~` - adjoint, alternating between a State Function and Measurement

This arithmetic, along with the Operators, allows quick and easy construction of many states for Quantum Algorithms. However, **one must be careful to use parentheses properly when performing arithmetic in this way.**

In [9]:
print(600 * ((One^5) + (Zero^5)))
print(One^Zero^3)

DictStateFn({'11111': 1.0, '00000': 1.0}) * 600.0
DictStateFn({'101010': 1})


States can also be easily converted between primitives. Below we'll see that we can also represent State functions by vectors and Quantum circuits. Note that to be consistent with the Operators, we use `to_matrix_op` to convert to a `VectorStateFn`.

In [10]:
print(((Plus^Minus)^2).to_matrix_op())
# TODO round
print(((Plus^One)^2).to_circuit_op())
print(((Plus^One)^2).to_matrix_op().sample())

VectorStateFn(Statevector([ 0.25-6.123234e-17j, -0.25+6.123234e-17j,  0.25-6.123234e-17j,
             -0.25+6.123234e-17j, -0.25+6.123234e-17j,  0.25-6.123234e-17j,
             -0.25+6.123234e-17j,  0.25-6.123234e-17j,  0.25-6.123234e-17j,
             -0.25+6.123234e-17j,  0.25-6.123234e-17j, -0.25+6.123234e-17j,
             -0.25+6.123234e-17j,  0.25-6.123234e-17j, -0.25+6.123234e-17j,
              0.25-6.123234e-17j],
            dims=(2, 2, 2, 2)))
CircuitStateFn(
     ┌───┐
q_0: ┤ X ├
     ├───┤
q_1: ┤ H ├
     ├───┤
q_2: ┤ X ├
     ├───┤
q_3: ┤ H ├
     └───┘
)
{'0111': 0.2744140625, '1101': 0.2529296875, '1111': 0.23828125, '0101': 0.234375}


In fact, `Plus` and `Minus` are `CircuitStateFn`s.

In [11]:
print(Plus)
print(Minus)

CircuitStateFn(
     ┌───┐
q_0: ┤ H ├
     └───┘
)
CircuitStateFn(
     ┌───┐┌───┐
q_0: ┤ X ├┤ H ├
     └───┘└───┘
)


Constructing a StateFn is easy. The `StateFn` class also serves as a factory, and can take any applicable primitive in its constructor and return the correct StateFn subclass. Right now the following primitives can be passed into the constructor, listed alongside the `StateFn` subclass they produce:

* str (equal to some basis bitstring) -> DictStateFn
* dict  -> DictStateFn
* Qiskit Result object -> DictStateFn
* list -> VectorStateFn
* np.ndarray -> VectorStateFn
* Statevector -> VectorStateFn
* QuantumCircuit -> CircuitStateFn
* Instruction -> CircuitStateFn
* OperatorBase -> OperatorStateFn

In [12]:
print(StateFn({'0':1}))
print(StateFn({'0':1}) == Zero)

print(StateFn([0,1,1,0]))

from qiskit.circuit.library import RealAmplitudes
print(StateFn(RealAmplitudes(2)))


DictStateFn({'0': 1})
True
VectorStateFn(Statevector([0.+0.j, 1.+0.j, 1.+0.j, 0.+0.j],
            dims=(2, 2)))
CircuitStateFn(
     ┌──────────┐     ┌──────────┐     ┌──────────┐     ┌──────────┐
q_0: ┤ RY(θ[0]) ├──■──┤ RY(θ[2]) ├──■──┤ RY(θ[4]) ├──■──┤ RY(θ[6]) ├
     ├──────────┤┌─┴─┐├──────────┤┌─┴─┐├──────────┤┌─┴─┐├──────────┤
q_1: ┤ RY(θ[1]) ├┤ X ├┤ RY(θ[3]) ├┤ X ├┤ RY(θ[5]) ├┤ X ├┤ RY(θ[7]) ├
     └──────────┘└───┘└──────────┘└───┘└──────────┘└───┘└──────────┘
)


## Part II: PrimitiveOps

Operators are functions which take StateFns to StateFns, and `PrimitiveOp`s are the building blocks for these Objects. For example, the identity Operator takes any StateFn to itself, and we can build this Operator efficiently out of `PauliOps`. Just like State Functions and Measurements, this characteristic function of the Operator is accessible via the `.eval` method, and the behavior of the function is defined by a computational primitive. If you pass a bitstring into an Operator's `eval` function, it will simply treat it as the corresponding basis StateFn.

Every `PrimitiveOp` has two properties:
* `primitive` - The data structure defining the underlying function's behavior. For `DictStateFn`, this object is a `dict`, etc.
* `coeff` - A coefficient multiplying the function, i.e. `StateFn(my_primitive, coeff=3) == StateFn(my_primitive) * 3`. Note that `coeff` can be int, float, complex or a free `Parameter` object (from `qiskit.circuit` in Terra) to be bound later.

Just like StateFn, `PrimitiveOp` is also a factory for creating the correct type of `PrimitiveOp` for a given primitive. Right now the following primitives can be passed into the constructor, listed alongside the `PrimitiveOp` subclass they produce:

* Terra's Pauli -> PauliOp
* Instruction -> CircuitOp
* QuantumCircuit -> CircuitOp
* 2d List -> MatrixOp
* np.ndarray -> MatrixOp
* spmatrix -> MatrixOp
* Terra's quantum_info.Operator -> MatrixOp

In [13]:
from qiskit.aqua.operators import X, Y, Z, I, CX, T, H, S, PrimitiveOp

In [14]:
print(X)
print(X.eval(One))
print(X.eval(One) == Zero)

print(CX)
print(CX.eval('01'))
print(CX.eval('01').eval('11'))
print(((~One^2) @ (CX.eval('01'))).eval())

print(((H^5) @ ((CX^2)^I) @ (I^(CX^2)))**2)
print((((H^5) @ ((CX^2)^I) @ (I^(CX^2)))**2) @ (Minus^5))
print(((H^I^I)@(X^I^I)@Zero))

X
DictStateFn({'0': (1+0j)})
True
          
q_0: ──■──
     ┌─┴─┐
q_1: ┤ X ├
     └───┘
VectorStateFn(Statevector([0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
            dims=(2, 2)))
(1+0j)
(1+0j)
          ┌───┐┌───┐     ┌───┐┌───┐
q_0: ──■──┤ I ├┤ H ├──■──┤ I ├┤ H ├
     ┌─┴─┐└───┘├───┤┌─┴─┐└───┘├───┤
q_1: ┤ X ├──■──┤ H ├┤ X ├──■──┤ H ├
     └───┘┌─┴─┐├───┤└───┘┌─┴─┐├───┤
q_2: ──■──┤ X ├┤ H ├──■──┤ X ├┤ H ├
     ┌─┴─┐└───┘├───┤┌─┴─┐└───┘├───┤
q_3: ┤ X ├──■──┤ H ├┤ X ├──■──┤ H ├
     └───┘┌─┴─┐├───┤└───┘┌─┴─┐├───┤
q_4: ─────┤ X ├┤ H ├─────┤ X ├┤ H ├
          └───┘└───┘     └───┘└───┘
CircuitStateFn(
     ┌───┐┌───┐     ┌───┐┌───┐     ┌───┐┌───┐
q_0: ┤ X ├┤ H ├──■──┤ I ├┤ H ├──■──┤ I ├┤ H ├
     ├───┤├───┤┌─┴─┐└───┘├───┤┌─┴─┐└───┘├───┤
q_1: ┤ X ├┤ H ├┤ X ├──■──┤ H ├┤ X ├──■──┤ H ├
     ├───┤├───┤└───┘┌─┴─┐├───┤└───┘┌─┴─┐├───┤
q_2: ┤ X ├┤ H ├──■──┤ X ├┤ H ├──■──┤ X ├┤ H ├
     ├───┤├───┤┌─┴─┐└───┘├───┤┌─┴─┐└───┘├───┤
q_3: ┤ X ├┤ H ├┤ X ├──■──┤ H ├┤ X ├──■──┤ H ├
     ├───┤├───┤└───┘┌─┴─┐├───┤└

## Part III: ListOps

If you've already played around with the above, you'll notice that you can easily perform operations between `StateFns` which we may not know how to perform efficiently in general (or simply haven't implemented an efficient procedure yet). In those cases, you may receive a `ListOp` result (or sublclass thereof) from your operation representing the lazy execution of the Operation. For example, if you attempt to add together a `DictStateFn` and a `CircuitStateFn`, you'll receive a `SummedOp` representing the sum of the two. This composite State function still has a working `eval`, but may need to perform a non-scalable computation under the hood, such as converting both to vectors.

Every `ListOp` has four properties:
* `oplist` - The list of `OperatorBase`s which define the behavior of the `ListOp`'s characteristic function.
* `combo_fn` - The function taking a list to an output value which defines how to combine the outputs of the `oplist` items' functions to produce the `ListOp`'s characteristic function.
* `coeff` - A coefficient multiplying the characteristic function.
* `abelian` - Indicates whether the Operators in `oplist` are known to mutually commute (usually set after being converted by the `AbelianGrouper` converter).

Note that `ListOp` supports typical iteration overloads, so you can use indexing like `my_op[4]` to access the `OperatorBase`s in `oplist`.

In [15]:
print(Zero + Plus)
print((Zero + Plus).eval('0'))

SummedOp(
[DictStateFn({'0': 1}),
CircuitStateFn(
     ┌───┐
q_0: ┤ H ├
     └───┘
)])
(1.7071067811865501+0j)


Or for composition, you might receive a `ComposedOp`. Note that just as in addition, we can sometimes perform the composition efficiently, such as between two circuit State functions. However, when composing a Measurement with a State function, we will always receive a `ComposedOp` back to reflect the fact that the State Function has been measured.

In [16]:
print('Dict measurement with Circuit statefn:')
print(~One @ Minus)
print((~One @ Minus).eval())
print('\nCircuit measurement with Circuit statefn:')
print(~One.to_circuit_op() @ Minus)
print('\nDict measurement with vector statefn:')
print(~One @ Minus.to_matrix_op())

Dict measurement with Circuit statefn:
ComposedOp(
[DictMeasurement({'1': 1}),
CircuitStateFn(
     ┌───┐┌───┐
q_0: ┤ X ├┤ H ├
     └───┘└───┘
)])
(-0.70710678118655+0j)

Circuit measurement with Circuit statefn:
ComposedOp(
[CircuitMeasurement(
     ┌───┐┌───┐┌───┐
q_0: ┤ X ├┤ H ├┤ X ├
     └───┘└───┘└───┘
),
DictStateFn({'0': 1})])

Dict measurement with vector statefn:
ComposedOp(
[DictMeasurement({'1': 1}),
VectorStateFn(Statevector([ 0.70710678-8.65956056e-17j, -0.70710678+8.65956056e-17j],
            dims=(2,)))])


#### Hierarchical ListOps

The base `ListOp` class is a special type of ListOp. Its `combo_fn` is the identity - it accepts a list and produces that same list. This allows the ListOp's characteristic function to represent the vectorization of the oplist's functions into a list, and the construction of large and interesting Operators.

In [17]:
print((~StateFn(Z) @ ListOp([One, Zero, Plus, Minus])).eval())
print(np.asarray((~StateFn(ListOp([Z, X])) @ ListOp([One, Zero, Plus, Minus])).eval()))

[(-1+0j), (1+0j), 0j, 0j]
[[-1.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  1.+0.j -1.-0.j]]


For example, building a Quantum Kernel is trivial using `ListOp`s.

In [18]:
kernel_states = ListOp([One, Zero, Plus, Minus])
print(np.asarray((~kernel_states @ kernel_states).eval()))

[[ 1.        +0.j  0.        +0.j  0.70710678+0.j -0.70710678+0.j]
 [ 0.        +0.j  1.        +0.j  0.70710678+0.j  0.70710678+0.j]
 [ 0.70710678+0.j  0.70710678+0.j  1.        +0.j  0.        +0.j]
 [-0.70710678-0.j  0.70710678+0.j  0.        +0.j  1.        +0.j]]


Note that in contrast with Terra, binding a list of values to a parameter in bind_parameters will return a ListOp of copies of the bound Operator with each parameterization.

In [19]:
from qiskit.circuit.library import ZZFeatureMap
feat_map = PrimitiveOp(ZZFeatureMap(2)) @ Zero
mapped_features = feat_map.bind_parameters(dict(zip(feat_map.primitive.ordered_parameters, np.eye(2).tolist())))
print(mapped_features)
qkernel_mat = (~mapped_features @ mapped_features).eval()
print(np.array(qkernel_mat))

ListOp(
[CircuitStateFn(
     ┌───┐┌───────┐                        ┌───┐┌───────┐                   »
q_0: ┤ H ├┤ U1(2) ├──■──────────────────■──┤ H ├┤ U1(2) ├──■────────────────»
     ├───┤├───────┤┌─┴─┐┌────────────┐┌─┴─┐├───┤├───────┤┌─┴─┐┌────────────┐»
q_1: ┤ H ├┤ U1(0) ├┤ X ├┤ U1(13.456) ├┤ X ├┤ H ├┤ U1(0) ├┤ X ├┤ U1(13.456) ├»
     └───┘└───────┘└───┘└────────────┘└───┘└───┘└───────┘└───┘└────────────┘»
«          
«q_0: ──■──
«     ┌─┴─┐
«q_1: ┤ X ├
«     └───┘
),
CircuitStateFn(
     ┌───┐┌───────┐                        ┌───┐┌───────┐                   »
q_0: ┤ H ├┤ U1(0) ├──■──────────────────■──┤ H ├┤ U1(0) ├──■────────────────»
     ├───┤├───────┤┌─┴─┐┌────────────┐┌─┴─┐├───┤├───────┤┌─┴─┐┌────────────┐»
q_1: ┤ H ├┤ U1(2) ├┤ X ├┤ U1(13.456) ├┤ X ├┤ H ├┤ U1(2) ├┤ X ├┤ U1(13.456) ├»
     └───┘└───────┘└───┘└────────────┘└───┘└───┘└───────┘└───┘└────────────┘»
«          
«q_0: ──■──
«     ┌─┴─┐
«q_1: ┤ X ├
«     └───┘
)])
[[ 1.        +0.j -0.20824985-0.j]
 [-0.20824985-0.j

## Part IV: Converters

Converters traverse an Operator structure and perform a particular manipulation or replacement of the Operators within. Typically, if a converter encounters an `OperatorBase` in the recursion which is irrelevant to its conversion purpose, that `OperatorBase` is left alone. Let's go back to our example from the top of this notebook to see the core converters in action.

In [20]:
import numpy as np
from qiskit.aqua.operators import I, X, Y, Z, H, CX, Zero, ListOp, PauliExpectation, PauliTrotterEvolution, CircuitSampler, MatrixEvolution
from qiskit.circuit import Parameter

### Evolutons, `exp_i()` and the `EvolvedOp`

Every `PrimitiveOp` and `ListOp` has an `.exp_i()` function to perform its Schrödinger-style exponentiation, $e^{-iHt}$. In practice, only a few of these Operators have an efficiently computable exponentiation (such as MatrixOp and the PauliOps with only one non-identity single-qubit Pauli), so we need to return a placeholder (similar to how `SummedOp` is a placeholder when we can't perform addition). This placeholder is called `EvolvedOp`, and it holds the Operator to be exponentiated in its `.primitive` property.

OpFlow fully supports parameterization, so we can use a `Parameter` for our evolution time here. Notice that there's no "evolution time" argument in any function. The Operator flow exponentiates whatever operator we tell it to, and if we choose to multiply the operator by an evolution time, $e^{-iHt}$, that will be reflected in our exponentiation parameters. This is not some trick to make it look like Physics - it actually works this way under the hood.

In [21]:
bell = CX @ (H^I) @ Zero
# We can also do CX @ (Plus ^ Zero)
evo_time = Parameter('θ')

evolution_op = (evo_time*two_qubit_H2).exp_i()
h2_measurement = StateFn(two_qubit_H2).adjoint()
evo_and_meas = h2_measurement @ evolution_op @ bell
print(evo_and_meas)

ComposedOp(
[OperatorMeasurement(SummedOp(
[-1.0523732 * II,
0.39793742 * IZ,
-0.3979374 * ZI,
-0.0112801 * ZZ,
0.18093119 * XX])),
e^(-i*1.0*θ * SummedOp(
[-1.0523732 * II,
0.39793742 * IZ,
-0.3979374 * ZI,
-0.0112801 * ZZ,
0.18093119 * XX])),
CircuitStateFn(
               
q_0: ───────■──
     ┌───┐┌─┴─┐
q_1: ┤ H ├┤ X ├
     └───┘└───┘
)])


We can use an `Evolution` converter to convert `EvolvedOp`s into `CircuitOp`s. The `PauliTrotterEvolution` will convert the Operator to be evolved into a sum of `PauliOp`s and use Trotterization to produce a circuit, and `MatrixEvolution` will convert to a `MatrixOp` and produce a circuit using Terra's `HamiltonianGate` (which is classical exponentiation under the hood). Note that evolution time parameter θ has flowed into the circuit.

In [22]:
trotterized_op = PauliTrotterEvolution(trotter_mode=Suzuki()).convert(evo_and_meas)
# We can also set trotter_mode using a string 'suzuki' or leave it empty to default to first order Trotterization.
print(trotterized_op)

ComposedOp(
[OperatorMeasurement(SummedOp(
[-1.0523732 * II,
0.39793742 * IZ,
-0.3979374 * ZI,
-0.0112801 * ZZ,
0.18093119 * XX])),
CircuitStateFn(
               ┌───┐┌───┐┌───────────────────┐┌───┐┌───┐┌───┐»
q_0: ───────■──┤ H ├┤ X ├┤ RZ(0.090465595*θ) ├┤ X ├┤ H ├┤ X ├»
     ┌───┐┌─┴─┐├───┤└─┬─┘└───────────────────┘└─┬─┘├───┤└─┬─┘»
q_1: ┤ H ├┤ X ├┤ H ├──■─────────────────────────■──┤ H ├──■──»
     └───┘└───┘└───┘                               └───┘     »
«     ┌───────────────────┐┌───┐┌──────────────────┐┌──────────────────┐┌───┐»
«q_0: ┤ RZ(-0.00564005*θ) ├┤ X ├┤ RZ(0.19896871*θ) ├┤ RZ(0.19896871*θ) ├┤ X ├»
«     └───────────────────┘└─┬─┘├──────────────────┤├──────────────────┤└─┬─┘»
«q_1: ───────────────────────■──┤ RZ(-0.1989687*θ) ├┤ RZ(-0.1989687*θ) ├──■──»
«                               └──────────────────┘└──────────────────┘     »
«     ┌───────────────────┐┌───┐┌───┐┌───┐┌───────────────────┐┌───┐┌───┐
«q_0: ┤ RZ(-0.00564005*θ) ├┤ X ├┤ H ├┤ X ├┤ RZ(0.090465595*θ) ├┤ X ├

We can bind our parameter to the operator if we so choose, and it will recursively bind into the circuit.

In [23]:
bound = trotterized_op.bind_parameters({evo_time: .5})
bound[1].to_circuit().draw(fold=1000)

┌───┐┌───┐┌──────────────┐┌───┐┌───┐┌───┐┌──────────────┐┌───┐ ┌──────────────┐ ┌──────────────┐┌───┐┌──────────────┐┌───┐┌───┐┌───┐┌──────────────┐┌───┐┌───┐
q_0: ───────■──┤ H ├┤ X ├┤ RZ(0.045233) ├┤ X ├┤ H ├┤ X ├┤ RZ(-0.00282) ├┤ X ├─┤ RZ(0.099484) ├─┤ RZ(0.099484) ├┤ X ├┤ RZ(-0.00282) ├┤ X ├┤ H ├┤ X ├┤ RZ(0.045233) ├┤ X ├┤ H ├
     ┌───┐┌─┴─┐├───┤└─┬─┘└──────────────┘└─┬─┘├───┤└─┬─┘└──────────────┘└─┬─┘┌┴──────────────┤┌┴──────────────┤└─┬─┘└──────────────┘└─┬─┘├───┤└─┬─┘└──────────────┘└─┬─┘├───┤
q_1: ┤ H ├┤ X ├┤ H ├──■────────────────────■──┤ H ├──■────────────────────■──┤ RZ(-0.099484) ├┤ RZ(-0.099484) ├──■────────────────────■──┤ H ├──■────────────────────■──┤ H ├
     └───┘└───┘└───┘                          └───┘                          └───────────────┘└───────────────┘                          └───┘                          └───┘

### Expectations for Converting Non-diagonal Measurements

In [24]:
diagonalized_meas_op = PauliExpectation().convert(trotterized_op)
print(diagonalized_meas_op)

SummedOp(
[ComposedOp(
[OperatorMeasurement(AbelianSummedOp(
[0.18093119 * ZZ,
-1.0523732 * II])),
CircuitStateFn(
               ┌───┐┌───┐┌───────────────────┐┌───┐┌───┐┌───┐»
q_0: ───────■──┤ H ├┤ X ├┤ RZ(0.090465595*θ) ├┤ X ├┤ H ├┤ X ├»
     ┌───┐┌─┴─┐├───┤└─┬─┘└───────────────────┘└─┬─┘├───┤└─┬─┘»
q_1: ┤ H ├┤ X ├┤ H ├──■─────────────────────────■──┤ H ├──■──»
     └───┘└───┘└───┘                               └───┘     »
«     ┌───────────────────┐┌───┐┌──────────────────┐┌──────────────────┐┌───┐»
«q_0: ┤ RZ(-0.00564005*θ) ├┤ X ├┤ RZ(0.19896871*θ) ├┤ RZ(0.19896871*θ) ├┤ X ├»
«     └───────────────────┘└─┬─┘├──────────────────┤├──────────────────┤└─┬─┘»
«q_1: ───────────────────────■──┤ RZ(-0.1989687*θ) ├┤ RZ(-0.1989687*θ) ├──■──»
«                               └──────────────────┘└──────────────────┘     »
«     ┌───────────────────┐┌───┐┌───┐┌───┐┌───────────────────┐┌───┐┌───┐┌───┐
«q_0: ┤ RZ(-0.00564005*θ) ├┤ X ├┤ H ├┤ X ├┤ RZ(0.090465595*θ) ├┤ X ├┤ H ├┤ H ├
«     └──────────

And now, we shard even further by binding with multiple parameters into a `ListOp`

In [25]:
evo_time_points = list(range(8))
h2_trotter_expectations = diagonalized_meas_op.bind_parameters({evo_time: evo_time_points})

### Executing `CircuitStateFn`s with the CircuitSampler

In [27]:
# Energies by Trotter expansion and sampled measurement
sampler = CircuitSampler(backend=BasicAer.get_backend('qasm_simulator'))
sampled_trotter_exp_op = sampler.convert(h2_trotter_expectations)
sampled_trotter_energies = sampled_trotter_exp_op.eval()
print('Sampled Trotterized energies:\n {}'.format(np.real(sampled_trotter_energies)))

Sampled Trotterized energies:
 [-1.05414011 -1.04283191 -1.03081694 -1.0354109  -1.06050097 -1.08135046
 -1.097606   -1.0937188   0.39074414  0.38448231  0.40539507  0.41738387
  0.38914564  0.35202137  0.29059272  0.13497217]
